In [1]:
import pandas as pd

In [17]:
data = pd.read_csv('science_questions.csv')

In [18]:
data.head()

,Unnamed: 0,questionID,AnswerKey,examName,schoolGrade,question,subject
0,0,AKDE&ED_2008_4_4,C,Alaska Department of Education & Early Develop...,4,Which organism needs to make its own food? (A)...,Biology
1,1,AKDE&ED_2008_4_31,D,Alaska Department of Education & Early Develop...,4,Students planted one hundred flower seeds. The...,Biology
2,2,AKDE&ED_2012_4_9,C,Alaska Department of Education & Early Develop...,4,Study the data table below. {Alaska-2012-4-10}...,Biology
3,3,AKDE&ED_2012_4_24,B,Alaska Department of Education & Early Develop...,4,The chart shows observations of the Moon. {Ala...,Biology
4,4,CSZ_2009_5_18,A,California Standards Test,5,A balloon has a negative charge. A glass rod h...,Biology


In [19]:
import nltk
import bs4 as bs
from nltk.tokenize import sent_tokenize # tokenizes sentences
import re
from nltk.stem import PorterStemmer
from nltk.tag import pos_tag
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer

In [20]:
def get_wordnet_pos(treebank_tag):

    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return 'n'

In [21]:
n, c = data.shape

In [22]:
n

2707

In [23]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
bag_question = []
for i in range(n):
    question = bs.BeautifulSoup(data['question'][i],features='lxml').text
    question = question.replace('.','. ')
    question = re.sub(r'\{.*\}', '', question)
    question = re.sub('[^a-zA-Z ]' ,'',question)
    question = question.lower()
    question_words = question.split()
    question_wo_stopwords = [w for w in question_words if not w in stopwords.words("english")]
    vect = CountVectorizer(max_features=5)
    if not question_wo_stopwords == []:
        vect.fit(question_wo_stopwords)
        bag = vect.transform(question_wo_stopwords)
        features = vect.get_feature_names()
        features.append(data['subject'][i])
        bag_question.append(features)
    if question_wo_stopwords == []:
        data = data.drop(i)

In [25]:
data.shape

(2699, 7)

In [26]:
data = data.reset_index(drop=True)

In [27]:
len(bag_question)

2699

In [28]:
bag_question

[['food', 'make', 'needs', 'organism', 'Biology'],
 ['could', 'flower', 'growth', 'hundred', 'observations', 'Biology'],
 ['bands', 'rubber', 'set', 'students', 'test', 'Biology'],
 ['chart', 'moon', 'observations', 'shows', 'Biology'],
 ['attracted', 'balloon', 'charge', 'glass', 'rod', 'Biology'],
 ['air', 'atmosphere', 'beach', 'bottom', 'diagram', 'Biology'],
 ['birds', 'characteristic', 'different', 'eat', 'feathers', 'Biology'],
 ['collected', 'description', 'following', 'rock', 'roger', 'Biology'],
 ['ago', 'animals', 'evidence', 'form', 'fossil', 'Biology'],
 ['actual', 'image', 'size', 'toy', 'truck', 'Biology'],
 ['animals', 'belongs', 'classifies', 'different', 'group', 'Biology'],
 ['container', 'lid', 'placed', 'table', 'water', 'Biology'],
 ['fan', 'image', 'mirror', 'table', 'window', 'Biology'],
 ['battery', 'bulb', 'circuit', 'light', 'terminal', 'Biology'],
 ['affected', 'drought', 'least', 'likely', 'little', 'Biology'],
 ['iron', 'magnetic', 'rocks', 'use', 'wand', 

In [29]:
data['bag_question'] = bag_question

In [30]:
data.head()

,Unnamed: 0,questionID,AnswerKey,examName,schoolGrade,question,subject,bag_question
0,0,AKDE&ED_2008_4_4,C,Alaska Department of Education & Early Develop...,4,Which organism needs to make its own food? (A)...,Biology,"[food, make, needs, organism, Biology]"
1,1,AKDE&ED_2008_4_31,D,Alaska Department of Education & Early Develop...,4,Students planted one hundred flower seeds. The...,Biology,"[could, flower, growth, hundred, observations,..."
2,2,AKDE&ED_2012_4_9,C,Alaska Department of Education & Early Develop...,4,Study the data table below. {Alaska-2012-4-10}...,Biology,"[bands, rubber, set, students, test, Biology]"
3,3,AKDE&ED_2012_4_24,B,Alaska Department of Education & Early Develop...,4,The chart shows observations of the Moon. {Ala...,Biology,"[chart, moon, observations, shows, Biology]"
4,4,CSZ_2009_5_18,A,California Standards Test,5,A balloon has a negative charge. A glass rod h...,Biology,"[attracted, balloon, charge, glass, rod, Biology]"


In [31]:
data.to_csv('science_questions_words.csv', index=False)